# Gathering Logs from Server and Portal


<img src="./imgs/captains-log.jpg"/>

Often times when working with server and portal, you need to exmine log files.  These files are accessed from the administration endpoints of the Pyton API.

### What are logs for?

- a historical record of everything and anything that happens within a system
- help track events on the system
- help administrators analyze problems
- detect common mistakes users make

### Accessing the Log Endpoints

For this notebook, I will be using an ArcGIS Enterprise configuration with multiple `servers`.

In [ ]:
from __future__ import annotations
from arcgis.gis import GIS
from arcgis.gis.admin import PortalAdminManager
from arcgis.gis.admin._logs import Logs
import datetime as _dt
from typing import Any
import pandas as pd

In [ ]:
gis:GIS = GIS(profile='your_dev_enterprise_profile')
admin:PortalAdminManager = gis.admin

#### Getting the Enterprise Logs

In [ ]:
logs:Logs =admin.logs
logs

#### Log Properties

The `admin.logs` enpoint provides access into the Enterprise's logs management system. The `Logs` can do the follow: `clean` or `query` the log files.

- clean: erases all the logs on the system
- query: allows administrators to examine/find log data

In [ ]:
start_date:_dt.datetime = _dt.datetime.now()
end_date:_dt.datetime = start_date - _dt.timedelta(days=2)
log_data:list = logs.query(
    start_time=start_date,
    end_time=end_date,
    page_size=1000,
)

In [ ]:
df = pd.DataFrame(data=log_data['logMessages'])

##### Example: Examining Source of Errors

In [ ]:
df.source.value_counts().plot(kind='barh')

Each log entry contains a wealth of information from where the error originated from to details on what caused the error.   

#### Gathering Server Logs

Since this enterprise configuration has multiple servers, a dictionary will be used to hold each `server`'s log manager object.

In [ ]:
admin_server_logs:dict = {server.url: server.logs for server in admin.servers.list()}

In [ ]:
frames:dict = {}
for k,log in admin_server_logs.items():
    print(f"Working on server: {k}")
    frames[k] = pd.DataFrame(data=log.query(start_time=start_date, end_time=end_date)['logMessages'])

In [ ]:
key:str = list(admin_server_logs.keys())[1]
frames[key].type.value_counts().plot(kind='bar')

## Wrapping up

The log endpoint provide a wealth of information. Explore your data, understand it, and it will help you solve problems on your configurations. 

<img src="./imgs/logwell.png"/>